# Setup Environment and Read Data

In [1]:
import torch
import numpy as np
import pandas as pd
import pickle
import copy
from tqdm import trange,tqdm
import torch.nn as nn
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score, classification_report, confusion_matrix

## Download the dataset

In [ ]:
!wget https://raw.githubusercontent.com/khundman/telemanom/master/labeled_anomalies.csv

--2025-04-21 14:04:03--  https://raw.githubusercontent.com/khundman/telemanom/master/labeled_anomalies.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3956 (3.9K) [text/plain]
Saving to: ‘labeled_anomalies.csv’

labeled_anomalies.c 100%[===================>]   3.86K  --.-KB/s    in 0s      

2025-04-21 14:04:04 (32.3 MB/s) - ‘labeled_anomalies.csv’ saved [3956/3956]



In [ ]:
%env DRIVE_PATH=/content/drive/MyDrive/Colab Notebooks/ELTE/DSLAB/

env: DRIVE_PATH=/content/drive/MyDrive/Colab Notebooks/ELTE/DSLAB/


In [ ]:
!mkdir "/root/.config/kaggle"

In [ ]:
!cp "$DRIVE_PATH/kaggle.json" "/root/.config/kaggle"
!chmod 600 "/root/.config/kaggle/kaggle.json"

In [ ]:
!cd "$DRIVE_PATH" && kaggle datasets download -d patrickfleith/nasa-anomaly-detection-dataset-smap-msl && mv nasa-anomaly-detection-dataset-smap-msl.zip data.zip && unzip -o data.zip && rm data.zip && mv data/data tmp && rm -r data && mv tmp data

Dataset URL: https://www.kaggle.com/datasets/patrickfleith/nasa-anomaly-detection-dataset-smap-msl
License(s): copyright-authors
Archive:  data.zip
  inflating: data/data/2018-05-19_15.00.10/models/A-1.h5  
  inflating: data/data/2018-05-19_15.00.10/models/A-2.h5  
  inflating: data/data/2018-05-19_15.00.10/models/A-3.h5  
  inflating: data/data/2018-05-19_15.00.10/models/A-4.h5  
  inflating: data/data/2018-05-19_15.00.10/models/A-5.h5  
  inflating: data/data/2018-05-19_15.00.10/models/A-6.h5  
  inflating: data/data/2018-05-19_15.00.10/models/A-7.h5  
  inflating: data/data/2018-05-19_15.00.10/models/A-8.h5  
  inflating: data/data/2018-05-19_15.00.10/models/A-9.h5  
  inflating: data/data/2018-05-19_15.00.10/models/B-1.h5  
  inflating: data/data/2018-05-19_15.00.10/models/C-1.h5  
  inflating: data/data/2018-05-19_15.00.10/models/C-2.h5  
  inflating: data/data/2018-05-19_15.00.10/models/D-1.h5  
  inflating: data/data/2018-05-19_15.00.10/models/D-11.h5  
  inflating: data/data/20

## Setup the dataset

In [2]:
DRIVE = "drive/MyDrive/Colab Notebooks/ELTE/DSLAB/ServerMachineDataset/"
DRIVE= "ServerMachineDataset/"
MACHINE = "machine-1-1.txt"
TRAIN_DATASET = DRIVE + "train/" + MACHINE
TEST_DATASET = DRIVE + "test/" + MACHINE
TEST_LABEL_DATASET = DRIVE + "test_label/" + MACHINE

metric = pd.read_csv(TRAIN_DATASET, header=None)
metric_test = pd.read_csv(TEST_DATASET, header=None)
true_anomalies = pd.read_csv(TEST_LABEL_DATASET, header=None)[0].to_numpy()

In [3]:
metric

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
0,0.032258,0.039195,0.027871,0.024390,0.0,0.915385,0.343691,0.0,0.020011,0.000122,...,0.0,0.004298,0.029993,0.022131,0.000000,0.000045,0.034677,0.034747,0.0,0.0
1,0.043011,0.048729,0.033445,0.025552,0.0,0.915385,0.344633,0.0,0.019160,0.001722,...,0.0,0.004298,0.030041,0.028821,0.000000,0.000045,0.035763,0.035833,0.0,0.0
2,0.043011,0.034958,0.032330,0.025552,0.0,0.915385,0.344633,0.0,0.020011,0.000122,...,0.0,0.004298,0.026248,0.021101,0.000000,0.000045,0.033012,0.033082,0.0,0.0
3,0.032258,0.028602,0.030100,0.024390,0.0,0.912821,0.342750,0.0,0.021289,0.000000,...,0.0,0.004298,0.030169,0.025733,0.000000,0.000022,0.035112,0.035182,0.0,0.0
4,0.032258,0.019068,0.026756,0.023229,0.0,0.912821,0.342750,0.0,0.018734,0.000000,...,0.0,0.004298,0.027240,0.022645,0.000000,0.000034,0.033447,0.033517,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28474,0.075269,0.046610,0.071349,0.076655,0.0,0.928205,0.269303,0.0,0.031649,0.000244,...,0.0,0.008596,0.068980,0.049408,0.000386,0.000034,0.064504,0.064572,0.0,0.0
28475,0.086022,0.070975,0.075808,0.077816,0.0,0.930769,0.269303,0.0,0.029946,0.000244,...,0.0,0.008596,0.073029,0.055584,0.000386,0.000034,0.067690,0.067757,0.0,0.0
28476,0.086022,0.065678,0.073579,0.076655,0.0,0.935897,0.270245,0.0,0.030372,0.000244,...,0.0,0.008596,0.070516,0.048893,0.000386,0.000034,0.064866,0.064934,0.0,0.0
28477,0.086022,0.056144,0.068004,0.074332,0.0,0.933333,0.271186,0.0,0.032643,0.000244,...,0.0,0.008596,0.070308,0.055069,0.000386,0.000045,0.067111,0.067178,0.0,0.0


# Preprocess the Dataset

In [ ]:
# Scale the values of the input metrics
scaler = MinMaxScaler()
metric_scaled = scaler.fit_transform(metric)
metric_tensor = torch.tensor(metric_scaled, dtype=torch.float32)
metric_scaled = pd.DataFrame(metric_scaled, index=metric.index, columns=metric.columns)
metric_scaled

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
0,0.065217,0.092965,0.100001,0.097221,0.0,0.915385,0.953003,0.0,0.169468,0.000869,...,0.0,0.150002,0.088565,0.052083,0.000000,0.035405,0.085685,0.085926,0.0,0.0
1,0.086957,0.115578,0.120001,0.101853,0.0,0.915385,0.955615,0.0,0.162261,0.012262,...,0.0,0.150002,0.088743,0.074651,0.000000,0.035405,0.089275,0.089517,0.0,0.0
2,0.086957,0.082915,0.116000,0.101853,0.0,0.915385,0.955615,0.0,0.169468,0.000869,...,0.0,0.150002,0.074656,0.048609,0.000000,0.035405,0.080180,0.080421,0.0,0.0
3,0.065217,0.067840,0.107999,0.097221,0.0,0.912821,0.950394,0.0,0.180291,0.000000,...,0.0,0.150002,0.089218,0.064234,0.000000,0.017309,0.087123,0.087364,0.0,0.0
4,0.065217,0.045227,0.096000,0.092593,0.0,0.912821,0.950394,0.0,0.158654,0.000000,...,0.0,0.150002,0.078340,0.053817,0.000000,0.026751,0.081618,0.081859,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28474,0.152174,0.110552,0.256000,0.305555,0.0,0.928205,0.746736,0.0,0.268028,0.001737,...,0.0,0.300003,0.233357,0.144096,0.166523,0.026751,0.184297,0.184538,0.0,0.0
28475,0.173914,0.168343,0.271999,0.310183,0.0,0.930769,0.746736,0.0,0.253606,0.001737,...,0.0,0.300003,0.248395,0.164929,0.166523,0.026751,0.194830,0.195069,0.0,0.0
28476,0.173914,0.155779,0.264001,0.305555,0.0,0.935897,0.749348,0.0,0.257213,0.001737,...,0.0,0.300003,0.239062,0.142359,0.166523,0.026751,0.185493,0.185735,0.0,0.0
28477,0.173914,0.133166,0.243998,0.296296,0.0,0.933333,0.751958,0.0,0.276446,0.001737,...,0.0,0.300003,0.238289,0.163192,0.166523,0.035405,0.192916,0.193155,0.0,0.0


### Scaled

In [4]:
# create train and test dataloaders
metric.interpolate(inplace=True)
metric.bfill(inplace=True)
metric_tensor = metric.values

metric_test.interpolate(inplace=True)
metric_test.bfill(inplace=True)
metric_test_tensor = metric_test.values

sequence_length = 30
sequences = []
for i in range(metric_tensor.shape[0] - sequence_length + 1):
  sequences.append(metric_tensor[i:i + sequence_length])


train_data, val_data = train_test_split(sequences, test_size=0.3, random_state=42) # 70% train, 30% temp

test_sequences = []
for i in range(metric_test_tensor.shape[0] - sequence_length + 1):
  test_sequences.append(metric_test_tensor[i:i + sequence_length])


batch_size = 32
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(dataset=test_sequences, batch_size=batch_size, shuffle=False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
sequences[0].shape

(30, 38)

# Define the Network

## LSTM

In [6]:
class LSTMEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim, num_layers=1):
        super(LSTMEncoder, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.fc_mean = nn.Linear(hidden_dim, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)

    def forward(self, x):
        _, (h_n, _) = self.lstm(x)  # h_n: (num_layers, batch, hidden_dim)
        h = h_n[-1]  # take the output of the last layer
        return self.fc_mean(h), self.fc_logvar(h)


class LSTMDecoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, output_dim, sequence_length, num_layers=1):
        super(LSTMDecoder, self).__init__()
        self.sequence_length = sequence_length
        self.latent_to_hidden = nn.Linear(latent_dim, hidden_dim)
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.output_layer = nn.Linear(hidden_dim, output_dim)

    def forward(self, z):
        # Repeat z for each timestep
        hidden = self.latent_to_hidden(z).unsqueeze(1).repeat(1, self.sequence_length, 1)
        out, _ = self.lstm(hidden)
        return self.output_layer(out)


class LSTMVAE(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim, sequence_length, num_layers=1, device='cpu'):
        super(LSTMVAE, self).__init__()
        self.encoder = LSTMEncoder(input_dim, hidden_dim, latent_dim, num_layers).to(device)
        self.decoder = LSTMDecoder(latent_dim, hidden_dim, input_dim, sequence_length, num_layers).to(device)

    def reparameterize(self, mean, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mean + eps * std

    def forward(self, x):
        mean, logvar = self.encoder(x)
        z = self.reparameterize(mean, logvar)
        x_recon = self.decoder(z)
        return x_recon, mean, logvar

In [7]:
input_dim = 38
hidden_dim = 128
latent_dim = 32
num_layers = 1

model = LSTMVAE(input_dim=input_dim,
                hidden_dim=hidden_dim,
                latent_dim=latent_dim,
                sequence_length=sequence_length,
                num_layers=num_layers,
                device=device).to(device)
optimizer = Adam(model.parameters(), lr=1e-3)

## Support functions

In [8]:
def loss_function(x, x_hat, mean, log_var):
    reproduction_loss = nn.functional.mse_loss(x_hat, x, reduction='sum')
    KLD = - 0.5 * torch.sum(1+ log_var - mean.pow(2) - log_var.exp())

    return reproduction_loss + KLD

In [9]:
def save_model(model):
    model_state = {
        'input_dim':input_dim,
        'latent_dim':latent_dim,
        'hidden_dim':hidden_dim,
        'state_dict':model.state_dict()
    }
    torch.save(model_state,'vae.pth')

# Train

## LSTM

In [10]:
torch.cuda.empty_cache()

scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.1)

# SPO optimizer - optuna
# bayesian hyperparameter tuning
# grid search - slow for DL

def train_model(model, train_loader, val_loader, optimizer, loss_fn, scheduler, num_epochs=10, device='cpu'):
    torch.cuda.empty_cache()
    train_losses = []
    val_losses = []

    early_stop_tolerant_count = 0
    early_stop_tolerant = 10
    best_loss = float('inf')
    best_model_wts = copy.deepcopy(model.state_dict())

    for epoch in range(num_epochs):
        train_loss = 0.0
        model.train()
        for batch in train_loader:
            batch = torch.tensor(batch, dtype=torch.float32).to(device)

            optimizer.zero_grad()

            recon_batch, mean, logvar = model(batch)
            loss = loss_fn(recon_batch, batch, mean, logvar)

            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        train_loss /= len(train_loader)
        train_losses.append(train_loss)

        # Validation
        model.eval()
        valid_loss = 0.0
        with torch.no_grad():
            for batch in val_loader:
                batch = torch.tensor(batch, dtype=torch.float32).to(device)
                recon_batch, mean, logvar = model(batch)
                loss = loss_fn(recon_batch, batch, mean, logvar)
                valid_loss += loss.item()

        valid_loss /= len(val_loader)
        val_losses.append(valid_loss)

        scheduler.step(valid_loss)

        if valid_loss < best_loss:
            best_loss = valid_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            early_stop_tolerant_count = 0
        else:
            early_stop_tolerant_count += 1

        print(f"Epoch {epoch+1:04d}: train loss {train_loss:.4f}, valid loss {valid_loss:.4f}")

        if early_stop_tolerant_count >= early_stop_tolerant:
            print("Early stopping triggered.")
            break

    model.load_state_dict(best_model_wts)
    print("Finished Training.")
    return train_losses, val_losses

#train_losses, val_losses = train_model(model, train_loader, val_loader, optimizer, loss_function, scheduler, num_epochs=100, device=device)

In [11]:
import optuna

def objective(trial):
    # Define hyperparameters to tune
    hidden_dim = trial.suggest_categorical("hidden_dim", [64, 128, 256])
    latent_dim = trial.suggest_categorical("latent_dim", [16, 32, 64])
    num_layers = trial.suggest_int("num_layers", 1, 3)
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-3)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])

    # Create DataLoader with the suggested batch size
    train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(dataset=val_data, batch_size=batch_size, shuffle=False)

    # Initialize model, optimizer, and scheduler with suggested hyperparameters
    model = LSTMVAE(input_dim=input_dim,
                    hidden_dim=hidden_dim,
                    latent_dim=latent_dim,
                    sequence_length=sequence_length,
                    num_layers=num_layers,
                    device=device).to(device)
    optimizer = Adam(model.parameters(), lr=learning_rate)
    scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.1)

    # Train the model (using a smaller number of epochs for tuning)
    train_losses, val_losses = train_model(model, train_loader, val_loader, optimizer, loss_function, scheduler, num_epochs=20, device=device)

    # Return the validation loss for Optuna to minimize
    return val_losses[-1]

# Run the Optuna study
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

# Print the best hyperparameters
print("Best hyperparameters: ", study.best_params)

C:\Users\beren\Desktop\notebooks\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-10-05 15:31:05,829] A new study created in memory with name: no-name-3c4fa056-ed4e-426a-90d0-a26aadda8bfb
C:\Users\beren\AppData\Local\Temp\ipykernel_5972\428586503.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-3)
C:\Users\beren\AppData\Local\Temp\ipykernel_5972\1313062830.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = torch.tens

Epoch 0001: train loss 476.5092, valid loss 188.7910
Epoch 0002: train loss 156.3893, valid loss 131.1115
Epoch 0003: train loss 115.2635, valid loss 104.6491
Epoch 0004: train loss 96.1119, valid loss 88.6886
Epoch 0005: train loss 81.3105, valid loss 75.0022
Epoch 0006: train loss 71.6718, valid loss 69.9957
Epoch 0007: train loss 67.3188, valid loss 66.4062
Epoch 0008: train loss 65.1471, valid loss 65.8117
Epoch 0009: train loss 63.7338, valid loss 64.1002
Epoch 0010: train loss 63.1082, valid loss 63.0475
Epoch 0011: train loss 62.8186, valid loss 64.2839
Epoch 0012: train loss 62.5271, valid loss 62.6748
Epoch 0013: train loss 62.5977, valid loss 63.1927
Epoch 0014: train loss 62.2373, valid loss 62.4420
Epoch 0015: train loss 62.3760, valid loss 63.2046
Epoch 0016: train loss 62.2536, valid loss 62.5954
Epoch 0017: train loss 62.0696, valid loss 62.6462
Epoch 0018: train loss 62.1868, valid loss 62.7016
Epoch 0019: train loss 61.9733, valid loss 62.0017


[I 2025-10-05 15:39:29,346] Trial 0 finished with value: 62.167745911673215 and parameters: {'hidden_dim': 256, 'latent_dim': 16, 'num_layers': 2, 'learning_rate': 1.532267139387774e-05, 'batch_size': 32}. Best is trial 0 with value: 62.167745911673215.


Epoch 0020: train loss 61.6651, valid loss 62.1677
Finished Training.
Epoch 0001: train loss 280.0942, valid loss 119.3896
Epoch 0002: train loss 98.5717, valid loss 82.6260
Epoch 0003: train loss 71.5608, valid loss 61.3643
Epoch 0004: train loss 55.1556, valid loss 51.1007
Epoch 0005: train loss 47.8795, valid loss 45.8650
Epoch 0006: train loss 44.3337, valid loss 43.1686
Epoch 0007: train loss 42.0228, valid loss 41.3335
Epoch 0008: train loss 40.7326, valid loss 40.7865
Epoch 0009: train loss 40.0880, valid loss 40.0295
Epoch 0010: train loss 40.0164, valid loss 39.3296
Epoch 0011: train loss 39.5045, valid loss 39.8813
Epoch 0012: train loss 39.2744, valid loss 39.2020
Epoch 0013: train loss 38.7457, valid loss 39.0126
Epoch 0014: train loss 38.4437, valid loss 38.6754
Epoch 0015: train loss 37.6230, valid loss 37.5876
Epoch 0016: train loss 36.6945, valid loss 35.8357
Epoch 0017: train loss 34.3270, valid loss 33.5787
Epoch 0018: train loss 32.7599, valid loss 32.5671
Epoch 0019

[I 2025-10-05 15:43:51,430] Trial 1 finished with value: 31.887346971347537 and parameters: {'hidden_dim': 128, 'latent_dim': 64, 'num_layers': 2, 'learning_rate': 1.633171211752766e-05, 'batch_size': 16}. Best is trial 1 with value: 31.887346971347537.


...

Epoch 0020: train loss 30.5077, valid loss 30.8408
Finished Training.
Epoch 0001: train loss 132.4852, valid loss 76.8057
Epoch 0002: train loss 66.2221, valid loss 70.4649
Epoch 0003: train loss 63.5639, valid loss 62.8397
Epoch 0004: train loss 62.8784, valid loss 64.8588
Epoch 0005: train loss 62.8502, valid loss 67.5388
Epoch 0006: train loss 62.5437, valid loss 61.8976
Epoch 0007: train loss 62.0948, valid loss 62.6066
Epoch 0008: train loss 62.0928, valid loss 62.6369
Epoch 0009: train loss 62.1902, valid loss 62.8973
Epoch 0010: train loss 62.2159, valid loss 64.6855
Epoch 0011: train loss 62.2694, valid loss 62.7934
Epoch 0012: train loss 62.0443, valid loss 62.7091
Epoch 0013: train loss 60.9741, valid loss 62.0512
Epoch 0014: train loss 60.8012, valid loss 61.3814
Epoch 0015: train loss 60.6681, valid loss 61.8465
Epoch 0016: train loss 60.5266, valid loss 61.5702
Epoch 0017: train loss 61.1593, valid loss 61.5576
Epoch 0018: train loss 60.8602, valid loss 61.8299
Epoch 0019:

[I 2025-10-05 18:55:45,246] Trial 48 finished with value: 62.19773724730988 and parameters: {'hidden_dim': 128, 'latent_dim': 32, 'num_layers': 3, 'learning_rate': 0.00043809017319436437, 'batch_size': 32}. Best is trial 34 with value: 30.610877956790425.


Epoch 0020: train loss 60.8481, valid loss 62.1977
Finished Training.
Epoch 0001: train loss 114.9127, valid loss 44.6689
Epoch 0002: train loss 40.3617, valid loss 38.0207
Epoch 0003: train loss 35.3235, valid loss 32.6977
Epoch 0004: train loss 32.0587, valid loss 32.1668
Epoch 0005: train loss 31.6050, valid loss 32.2038
Epoch 0006: train loss 31.4338, valid loss 31.3723
Epoch 0007: train loss 31.3153, valid loss 31.1373
Epoch 0008: train loss 31.0762, valid loss 31.4471
Epoch 0009: train loss 30.8554, valid loss 31.3886
Epoch 0010: train loss 30.9475, valid loss 31.2266
Epoch 0011: train loss 30.9536, valid loss 31.2991
Epoch 0012: train loss 30.8904, valid loss 31.0253
Epoch 0013: train loss 30.8637, valid loss 31.1443
Epoch 0014: train loss 30.8948, valid loss 31.9729
Epoch 0015: train loss 30.8221, valid loss 31.1542
Epoch 0016: train loss 30.7399, valid loss 30.8739
Epoch 0017: train loss 30.6709, valid loss 31.1048
Epoch 0018: train loss 30.9284, valid loss 31.3669
Epoch 0019:

[I 2025-10-05 18:59:58,238] Trial 49 finished with value: 30.97142788176233 and parameters: {'hidden_dim': 64, 'latent_dim': 32, 'num_layers': 3, 'learning_rate': 0.00013959292792355463, 'batch_size': 16}. Best is trial 34 with value: 30.610877956790425.


Epoch 0020: train loss 30.8193, valid loss 30.9714
Finished Training.
Best hyperparameters:  {'hidden_dim': 64, 'latent_dim': 32, 'num_layers': 3, 'learning_rate': 0.0002398370047454381, 'batch_size': 16}


In [24]:
import joblib

joblib.dump(study, "study.pkl")   # save study

# to load it:
jl = joblib.load("study.pkl")

print(jl.best_trial.params)

{'hidden_dim': 64, 'latent_dim': 32, 'num_layers': 3, 'learning_rate': 0.0002398370047454381, 'batch_size': 16}


# Evaluate

In [ ]:
def evaluate_lstm(model, test_loader, device, percentile_threshold=90):
    model.eval()
    anomaly_scores = []

    with torch.no_grad():
        for batch in test_loader:
            batch = torch.tensor(batch, dtype=torch.float32).to(device)

            batch_scores = []
            for i in range(batch.shape[0]): #Iterate through each sequence in the batch
                sequence = batch[i, :, :].unsqueeze(0)  # Select a single sequence
                recon_batch, mean, logvar = model(sequence)
                loss = loss_function(recon_batch, sequence, mean, logvar)
                batch_scores.append(loss.item())
            anomaly_scores.extend(batch_scores)  # Append scores for all sequences in the batch


    # Calculate the threshold based on the specified percentile
    threshold = np.percentile(anomaly_scores, percentile_threshold)

    # Identify anomaly indices
    anomaly_indices = [i for i, score in enumerate(anomaly_scores) if score > threshold]
    return anomaly_indices
anomalies = evaluate_lstm(model, test_loader, device, 90)

In [ ]:
def calculate_f1_score(anomaly_indices, true_anomalies):
    # Create a binary array representing predicted anomalies
    predicted_anomalies = np.zeros_like(true_anomalies)
    for index in anomaly_indices:
        if index < len(predicted_anomalies):  # Check index bounds
          predicted_anomalies[index] = 1

    # Calculate the F1 score
    f1 = f1_score(true_anomalies, predicted_anomalies)
    return f1, predicted_anomalies

# Example usage (assuming 'anomalies' and 'true_anomalies' are defined)
f1, predicted_anomalies = calculate_f1_score(anomalies, true_anomalies)
print(f"F1 Score: {f1}")

In [ ]:
print(classification_report(true_anomalies, predicted_anomalies))

In [ ]:
print(confusion_matrix(true_anomalies, predicted_anomalies))